In [2]:
import utility_functions as utils
import importlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
import itertools
from collections import Counter
import os
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
from nltk.corpus import stopwords as stop_words

importlib.reload(utils)
path_to_east_coast_lyrics = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Lyrics/east_coast'
path_to_west_coast_lyrics = '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/Lyrics/west_coast'

concreteness_df = pd.read_excel('./concreteness_scores.xlsx')
concreteness_ratings = concreteness_df.set_index('Word')['Conc.M'].to_dict()

importlib.reload(utils)

/Users/borosabel/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/borosabel/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/borosabel/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


<module 'utility_functions' from '/Users/borosabel/Documents/Uni/Thesis/PopMIR/Code/Lyrics/utility_functions.py'>

In [3]:
# Load data into dataframes
east_coast_df = utils.load_txt_into_dataframe(path_to_east_coast_lyrics)
west_coast_df = utils.load_txt_into_dataframe(path_to_west_coast_lyrics)

In [4]:
# Clean up lyrics column
east_coast_df['Lyrics'] = east_coast_df['Lyrics'].apply(utils.cleanup)
west_coast_df['Lyrics'] = west_coast_df['Lyrics'].apply(utils.cleanup)

In [5]:
# Assuming 'df' is your DataFrame and 'text_column' is the name of the column containing text
west_coast_df.to_csv('west_coast_lyrics_line_by_line.txt', columns=['Lyrics'], index=False, header=False, encoding='utf-8')

In [6]:
text_file = 'west_coast_lyrics_line_by_line.txt'
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:2000]]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [7]:
tp = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

/Users/borosabel/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/borosabel/lib/python3.9/site-packages/contextualized_topic_models/utils/data_preparation.py:64: UserWarning: the longest document in your collection has 1282 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 4/4 [00:12<00:00,  3.04s/it]


In [11]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=5, num_epochs=10)
ctm.fit(training_dataset) # run the model

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

In [13]:
ctm.save(models_dir="./")

/Users/borosabel/lib/python3.9/site-packages/contextualized_topic_models/models/ctm.py:640: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(


In [12]:
ctm.get_topic_lists(5)

[['wax', 'hoping', 'shoe', 'rapping', 'phony'],
 ['ghetto', 'heaven', 'know', 'bye', 'big'],
 ['flavor', 'equipped', 'grind', 'startin', 'wise'],
 ['go', 'nigga', 'come', 'see', 'cause'],
 ['nigga', 'know', 'wit', 'like', 'dick']]

In [10]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

  0%|          | 0/11 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true